# M6.19 Titanic: Machine Learning Disaster 
### Group 4 - Baseline Submission
#### Team Members:
* Marc Mahanna,
* Nicholas Vitellaro,
* Makenna Owens,
* Jonathan Wang

In [118]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

In [119]:
# Load Data
# train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data = pd.read_csv("../Data/train.csv")
# displaying first 5 rows of the dataset
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [120]:
# test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data = pd.read_csv("../Data/test.csv")
# displaying first 5 rows of the test dataset
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [121]:
# Feature Engineering
# Fill missing values for 'Age' with median, 'Embarked' with mode, and 'Fare' with median
train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
train_data['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)
test_data['Age'].fillna(test_data['Age'].median(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].median(), inplace=True)

# Convert categorical features and engineer new features
train_data['FamilySize'] = train_data['SibSp'] + train_data['Parch']
test_data['FamilySize'] = test_data['SibSp'] + test_data['Parch']

/var/folders/y5/54wq6fj15m90x1g7cxplt8hr0000gn/T/ipykernel_19502/241271474.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data['Age'].fillna(train_data['Age'].median(), inplace=True)
/var/folders/y5/54wq6fj15m90x1g7cxplt8hr0000gn/T/ipykernel_19502/241271474.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are set

In [122]:
# Select features and one-hot encode categorical variables
features = ["Pclass", "Sex", "SibSp", "Parch", "Fare", "Embarked", "FamilySize", "Age"]
# features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])
y = train_data["Survived"]

In [123]:
param_grid = {'min_samples_leaf': [1, 2, 4], 'max_depth': [3, 5, 7, 9, 11, 13, 15],
              'max_features': [10, 20, 30, 40, 60, 80, 100],
              'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900]}
grid_search = GridSearchCV(RandomForestClassifier(random_state=1),
                           param_grid,
                           cv=5,
                           scoring='accuracy',
                           verbose=3,
                           n_jobs=-1)
grid_search.fit(X, y)

model = grid_search.best_estimator_
score = grid_search.best_score_
print("Best score is: ", score)

Fitting 5 folds for each of 1323 candidates, totalling 6615 fits
Best score is:  0.8406565815077522


/opt/anaconda3/envs/cosc523_proj/lib/python3.10/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [124]:
# making predictions on test dataset
predictions = model.predict(X_test)

In [125]:
# creating df with survival predictions
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})

# saving output to csv 
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
